# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
M. Schirmer  ->  M. Schirmer  |  ['M. Schirmer']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
Arxiv has 61 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2305.09721


extracting tarball to tmp_2305.09721...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure figs/PS_EagIllTNGTNG-NR_kbinned_fiducial_cifog_20pcnt_fesc_zedk20.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:125: LatexWarning: Recovered figure figs/PS_EagIllTNGTNG-NR_kbinned_fiducial_cifog_20pcnt_fesc_zedk20.pdf as tmp_2305.09721/./figs/PS_EagIllTNGTNG-NR_kbinned_fiducial_cifog_20pcnt_fesc_zedk20.pdf.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


...

✔ → 202066:\section{Non-exhaustive list of publications about contamination in spacecraft\label{sec:apdx-library}}
  ↳ 210398:\section{Pertinent renderings and photographs\label{sec:apdx_photos}}
✔ → 210398:\section{Pertinent renderings and photographs\label{sec:apdx_photos}}
  ↳ 214546:\section{Tables}


✘ → 214546:\section{Tables}
  ↳ 216829:\section{\label{apdx_contamination_M1M2}Contamination estimates for the telescope cavity (mirrors M1 and M2)}


✔ → 216829:\section{\label{apdx_contamination_M1M2}Contamination estimates for the telescope cavity (mirrors M1 and M2)}
  ↳ 226485:\section{\label{apdx_contamination_NISPVIS}Contamination estimation for the instrument cavity}


✔ → 226485:\section{\label{apdx_contamination_NISPVIS}Contamination estimation for the instrument cavity}
  ↳ 228916:end


list index out of range


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2305.09721-b31b1b.svg)](https://arxiv.org/abs/arXiv:2305.09721) | **The signature of galaxy formation models in the power spectrum of the  hydrogen 21cm line during reionization**  |
|| J. S. W. Lewis, et al. -- incl., <mark>A. Pillepich</mark> |
|*Appeared on*| *2023-05-18*|
|*Comments*| *submitted to MNRAS (12/05/23)*|
|**Abstract**| Observations of the 21cm line of neutral hydrogen are poised to revolutionize our knowledge of cosmic reionization and the high-redshift population of galaxies. However, harnessing such information requires robust and comprehensive theoretical modeling. We study the non-linear effects of hydrodynamics and astrophysical feedback processes, including stellar and AGN feedback, on the 21cm signal by post-processing three existing cosmological hydrodynamical simulations of galaxy formation: Illustris, IllustrisTNG, and Eagle. Overall and despite their different underlying galaxy-formation models, the three simulations return similar predictions for the global 21cm rightness temperature and its power spectrum. At fixed redshift, most differences are attributable to differences in the history of reionization, in turn driven by differences in the build-up of stellar sources of radiation. However, the impact of astrophysics is imprinted in the 21cm power spectrum through several unique signatures. First, we find significant small scale ($k \geq 10\, \rm {Mpc}^{-1}$) differences between Illustris and IllustrisTNG, where higher velocity winds generated by supernova feedback soften density peaks and lead to lower 21cm power in TNG. Second, we find more 21cm power at intermediate scales ($k \approx 0.8\, \rm {Mpc}^{-1}$) in Eagle, due to differences in ionization driven by highly effective stellar feedback, leading to lower star formation, older and redder stellar populations, and thus lower ionizing luminosities. Though subtle, these features could allow future observations of the 21cm signal, in conjunction with other reionization observables, to constrain theoretical models for galactic feedback at high redshift. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2305.10107-b31b1b.svg)](https://arxiv.org/abs/arXiv:2305.10107) | **Euclid preparation. XXIX. Water ice in spacecraft part I: The physics of  ice formation and contamination**  |
|| E. Collaboration, et al. -- incl., <mark>M. Schirmer</mark> |
|*Appeared on*| *2023-05-18*|
|*Comments*| *35 pages, 22 figures, accepted for publication in A&A*|
|**Abstract**| Molecular contamination is a well-known problem in space flight. Water is the most common contaminant and alters numerous properties of a cryogenic optical system. Too much ice means that Euclid's calibration requirements and science goals cannot be met. Euclid must then be thermally decontaminated, a long and risky process. We need to understand how iced optics affect the data and when a decontamination is required. This is essential to build adequate calibration and survey plans, yet a comprehensive analysis in the context of an astrophysical space survey has not been done before. In this paper we look at other spacecraft with well-documented outgassing records, and we review the formation of thin ice films. A mix of amorphous and crystalline ices is expected for Euclid. Their surface topography depends on the competing energetic needs of the substrate-water and the water-water interfaces, and is hard to predict with current theories. We illustrate that with scanning-tunnelling and atomic-force microscope images. Industrial tools exist to estimate contamination, and we must understand their uncertainties. We find considerable knowledge errors on the diffusion and sublimation coefficients, limiting the accuracy of these tools. We developed a water transport model to compute contamination rates in Euclid, and find general agreement with industry estimates. Tests of the Euclid flight hardware in space simulators did not pick up contamination signals; our in-flight calibrations observations will be much more sensitive. We must understand the link between the amount of ice on the optics and its effect on Euclid's data. Little research is available about this link, possibly because other spacecraft can decontaminate easily, quenching the need for a deeper understanding. In our second paper we quantify the various effects of iced optics on spectrophotometric data. |

## Failed papers

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2305.09721.md
    + _build/html/tmp_2305.09721/./figs/PS_TNG_TNG_Ill_comparison_xhi600_fiducial_cifog_20pcnt_fesc_cifog_20pcnt_fesc_Illustris-1_ratio.png
    + _build/html/tmp_2305.09721/./figs/main_panel/PS_EagIllTNGTNG-NR_comparison_z780_fiducialfiducial_cifog_20pcnt_fesc.png
    + _build/html/tmp_2305.09721/./figs/main_panel/PS_EagIllTNGTNG-NR_comparison_z630_fiducialfiducial_cifog_20pcnt_fesc.png
    + _build/html/tmp_2305.09721/./figs/main_panel/PS_EagIllTNGTNG-NR_comparison_z570_fiducialfiducial_cifog_20pcnt_fesc.png
    + _build/html/tmp_2305.09721/./figs/main_panel/PS_EagIllTNGTNG-NR_comparison_xhi850_fiducialfiducial_cifog_20pcnt_fesc.png
    + _build/html/tmp_2305.09721/./figs/main_panel/PS_EagIllTNGTNG-NR_comparison_xhi600_fiducialfiducial_cifog_20pcnt_fesc.png
    + _build/html/tmp_2305.09721/./figs/main_panel/PS_EagIllTNGTNG-NR_comparison_xhi300_fiducialfiducial_cifog_20pcnt_fesc.png
    + _build/html/tmp_2305.09721/./figs/main_panel/PS_EagIllTNGTN

## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\fesc}{f_{\rm esc}}$
$\newcommand{\xhi}{x_{\rm HI}}$
$\newcommand{\mstel}{M_\star}$
$\newcommand{\lintr}{L_{\rm intr}}$
$\newcommand{\lesc}{L_{\rm esc}}$
$\newcommand{\fescglob}{f_{\rm esc}^{20\%}}$
$\newcommand{\fescevol}{f_{\rm esc}(z)}$
$\newcommand{\feschalo}{f_{\rm esc}(M_{\rm h})}$
$\newcommand{\mean}[1]{\langle #1 \rangle}$
$\newcommand{\dtb}{\delta T_{\rm b}}$
$\newcommand{\avgdtb}{\overline{\delta T_{\rm b}}}$
$\newcommand{\tcmb}{T_{\rm CMB}}$
$\newcommand{\dtbPS}{\overline{\delta T_{\rm b}^2}  \Delta_{21}^2}$
$\newcommand{\PS}{\Delta_{21}^2}$
$\newcommand{\figref}[1]{Fig. \ref{#1}}$
$\newcommand{\code}[1]{\textsc{#1}}$
$\newcommand{\jl}[1]{\noindent{\color{red}[Joe: #1]}}$
$\newcommand{\dn}[1]{\noindent{\bf \color{violet}[Dylan: #1]}}$
$\newcommand{\rk}[1]{\noindent{\bf \color{cyan}[Ralf: #1]}}$
$\newcommand{\sg}[1]{\noindent{\bf \color{orange}[Simon: #1]}}$
$\newcommand{\ap}[1]{{\color{magenta} #1}}$</div>



<div id="title">

# The signature of galaxy formation models in the power spectrum of the hydrogen 21cm line during reionization

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2305.09721-b31b1b.svg)](https://arxiv.org/abs/2305.09721)<mark>Appeared on: 2023-05-18</mark> -  _submitted to MNRAS (12/05/23)_

</div>
<div id="authors">

J. S. W. Lewis, et al. -- incl., <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** Observations of the 21cm line of neutral hydrogen are poised to revolutionize our knowledge of cosmic reionization and the high-redshift population of galaxies. However, harnessing such information requires robust and comprehensive theoretical modeling. We study the non-linear effects of hydrodynamics and astrophysical feedback processes, including stellar and AGN feedback, on the 21cm signal by post-processing three existing cosmological hydrodynamical simulations of galaxy formation: Illustris, IllustrisTNG, and Eagle. Overall and despite their different underlying galaxy-formation models, the three simulations return similar predictions for the global 21cm rightness temperature and its power spectrum. At fixed redshift, most differences are attributable to differences in the history of reionization, in turn driven by differences in the build-up of stellar sources of radiation. However, the impact of astrophysics is imprinted in the 21cm power spectrum through several unique signatures. First, we find significant small scale ( $k \geq 10  \rm{Mpc}^{-1}$ ) differences between Illustris and IllustrisTNG, where higher velocity winds generated by supernova feedback soften density peaks and lead to lower 21cm power in TNG. Second, we find more 21cm power at intermediate scales ( $k \approx 0.8  \rm{Mpc}^{-1}$ ) in Eagle, due to differences in ionization driven by highly effective stellar feedback, leading to lower star formation, older and redder stellar populations, and thus lower ionizing luminosities. Though subtle, these features could allow future observations of the 21cm signal, in conjunction with other reionization observables, to constrain theoretical models for galactic feedback at high redshift.

</div>

<div id="div_fig1">

<img src="tmp_2305.09721/./figs/PS_TNG_TNG_Ill_comparison_xhi600_fiducial_cifog_20pcnt_fesc_cifog_20pcnt_fesc_Illustris-1_ratio.png" alt="Fig1" width="100%"/>

**Figure 1. -** Relative 21cm power spectra for an experimental analysis of the TNG100 simulation, keeping all fiducial choices except we use the sources of ionizing radiation as predicted by Illustris. This test run is called "Illustris sources" (dashed dotted curves) and is compared to the fiducial Illustris and TNG100 predictions via the ratio of the 21cm power spectrum to TNG100. Each power spectrum is taken at an epoch where average $\xhi$$  $\ap$prox 0.6$. The differences in the 21cm power spectrum at $k \leq  $\SI{7}{\per\mega\parsec} in Illustris vs. TNG100 are caused by differences in the stellar populations rather than in the gas state. Thinner curves denote scales where the predictions are affected by resolution or box size. (*fig:ill_src_ps*)

</div>
<div id="div_fig2">

<img src="tmp_2305.09721/./figs/main_panel/PS_EagIllTNGTNG-NR_comparison_z780_fiducialfiducial_cifog_20pcnt_fesc.png" alt="Fig7.1" width="11%"/><img src="tmp_2305.09721/./figs/main_panel/PS_EagIllTNGTNG-NR_comparison_z630_fiducialfiducial_cifog_20pcnt_fesc.png" alt="Fig7.2" width="11%"/><img src="tmp_2305.09721/./figs/main_panel/PS_EagIllTNGTNG-NR_comparison_z570_fiducialfiducial_cifog_20pcnt_fesc.png" alt="Fig7.3" width="11%"/><img src="tmp_2305.09721/./figs/main_panel/PS_EagIllTNGTNG-NR_comparison_xhi850_fiducialfiducial_cifog_20pcnt_fesc.png" alt="Fig7.4" width="11%"/><img src="tmp_2305.09721/./figs/main_panel/PS_EagIllTNGTNG-NR_comparison_xhi600_fiducialfiducial_cifog_20pcnt_fesc.png" alt="Fig7.5" width="11%"/><img src="tmp_2305.09721/./figs/main_panel/PS_EagIllTNGTNG-NR_comparison_xhi300_fiducialfiducial_cifog_20pcnt_fesc.png" alt="Fig7.6" width="11%"/><img src="tmp_2305.09721/./figs/main_panel/PS_EagIllTNGTNG-NR_comparison_xhi850_fiducialfiducial_cifog_20pcnt_fesc_ratio.png" alt="Fig7.7" width="11%"/><img src="tmp_2305.09721/./figs/main_panel/PS_EagIllTNGTNG-NR_comparison_xhi600_fiducialfiducial_cifog_20pcnt_fesc_ratio.png" alt="Fig7.8" width="11%"/><img src="tmp_2305.09721/./figs/main_panel/PS_EagIllTNGTNG-NR_comparison_xhi300_fiducialfiducial_cifog_20pcnt_fesc_ratio.png" alt="Fig7.9" width="11%"/>

**Figure 7. -** Direct comparison of the power spectra of the 21cm brightness temperature, $\dtb$PS , from TNG100, Illustris, Eagle, and TNG100-NR across time. Coloured curves show the fiducial results, whereas the shaded areas indicate the spread of results using different post-processing models for $\dtb$(Section \ref{sec:fiducial} and Appendix \ref{app:model_var}). Thinner curves denote scales where the predictions are affected by resolution or box size. **Top row:** power spectra for all four simulations at fixed redshifts. **Middle row:** power spectra grouped so that the average neutral fraction ($\xhi$) in every panel is similar. **Bottom row:** Same as the middle row, but normalising all power spectra by the TNG100 result to quantify the level of (dis)agreement. (*fig:PS_compare*)

</div>
<div id="div_fig3">

<img src="tmp_2305.09721/./figs/decomp/decomp_TNG100-1TNG100-1-NREagle100-1Illustris-1_fiducialfiducial_cifog_20pcnt_fesc_xhi850.png" alt="Fig8.1" width="33%"/><img src="tmp_2305.09721/./figs/decomp/decomp_TNG100-1TNG100-1-NREagle100-1Illustris-1_fiducialfiducial_cifog_20pcnt_fesc_xhi600.png" alt="Fig8.2" width="33%"/><img src="tmp_2305.09721/./figs/decomp/decomp_TNG100-1TNG100-1-NREagle100-1Illustris-1_fiducialfiducial_cifog_20pcnt_fesc_xhi300.png" alt="Fig8.3" width="33%"/>

**Figure 8. -** Normalized 21cm power spectrum, $ \Delta^2_{21}$, in our fiducial post-processing model and its contributing terms (as detailed in Eq. \ref{eq:dTb2}), for all simulations. To simplify this plot, we do not show any constituent cross terms of $ \Delta^2_{21}$, which can however contribute significantly at some scales and ionization levels \protect\citep[e.g.][]{georgiev_large-scale_2022}. We have grouped the simulations at redshifts when their average ionization fractions are similar. Solid thick curves show the fiducial results. Dashed curves denote the approximate contribution to $ \Delta^2_{21}$ from the gas density term, whereas dotted and solid thin curves represent the $\xhi$  term and the approximate contribution of the redshift-space velocity distortions (velocity term), respectively. (*fig:ps_ingredients*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2305.09721"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\}{natexlab}$</div>



<div id="title">

# $\Euclid$ preparation. XXIX. Water ice in spacecraft part I:$\The physics of ice formation and contamination$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2305.10107-b31b1b.svg)](https://arxiv.org/abs/2305.10107)<mark>Appeared on: 2023-05-18</mark> -  _35 pages, 22 figures, accepted for publication in A&A_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>M. Schirmer</mark>

</div>
<div id="abstract">

**Abstract:** Material outgassing in a vacuum leads to molecular contamination, a well-known problem in spaceflight. Water is the most common contaminant in cryogenic spacecraft, altering numerous properties of optical systems. Too much ice means that $\Euclid$ 's calibration requirements cannot be met anymore. $\Euclid$ must then be thermally decontaminated, a month-long risky operation. We need to understand how ice affects our data to build adequate calibration and survey plans. A comprehensive analysis in the context of an astrophysical space survey has not been done before.   In this paper we look at other spacecraft with well-documented outgassing records. We then review the formation of thin ice films, and find that for $\Euclid$ a mix of amorphous and crystalline ices is expected. Their surface topography -- and thus optical properties -- depend on the competing energetic needs of the substrate-water and the water-water interfaces, and are hard to predict with current theories. We illustrate that with scanning-tunnelling and atomic-force microscope images of thin ice films.   Sophisticated tools exist to compute contamination rates, and we must understand their underlying physical principles and uncertainties. We find considerable knowledge errors on the diffusion and sublimation coefficients, limiting the accuracy of outgassing estimates. We developed a water transport model to compute contamination rates in $\Euclid$ , and find agreement with industry estimates within the uncertainties. Tests of the $\Euclid$ flight hardware in space simulators did not pick up significant contamination signals, but were also not geared towards this purpose; our in-flight calibrations observations will be much more sensitive.   To derive a calibration and decontamination strategy we need to understand the link between the amount of ice in the optics and its effect on the data. There is little research about this, possibly because other spacecraft can decontaminate more easily, quenching the need for a deeper understanding. In our second paper we quantify the impact of iced optics on $\Euclid$ 's data.

</div>

<div id="div_fig1">

<img src="tmp_2305.10107/./sublimation_curve.png" alt="Fig11" width="100%"/>

**Figure 11. -** Sublimation-flux models for amorphous and crystalline (hexagonal) ice. Overlaid are various measurements. The model for amorphous ice is shown up to 120 K by the dashed pink line; it is obtained by shifting the \cite{murphy2005} curve by 3 K to lower temperatures. (*fig:sublimation_rate*)

</div>
<div id="div_fig2">

<img src="tmp_2305.10107/./plm_contamination_new.png" alt="Fig21" width="100%"/>

**Figure 21. -** Geometric model of the telescope to compute contamination on the optical surfaces. _Left panel_: Contamination from a point $Q$ on the baffle wall onto a point $P$ on M1 (bottom grey disk). _Right panel_: Contamination from a point $Q$ on M2 (top grey disk) onto $P$. The same setup can be used to compute contamination of M1 by sublimation from the `front ring' (FR), contamination of M2 from M1 and the `back ring' BR, and the water flux loss through the front aperture, simply by adjusting the respective integral bounds. The blue lines indicate fixed parameters listed in Table \ref{plm_model_dimensions}. (*plm_model*)

</div>
<div id="div_fig3">

<img src="tmp_2305.10107/./crystallization_kouchi.png" alt="Fig8" width="100%"/>

**Figure 8. -** Annealing time for amorphous ice \Ial to reach different fractions of crystallisation, using the \cite{kouchi1994} formalism that is based on kinetic theory of crystallisation. The shaded box shows the relevant time and temperate ranges for \Euclid. The crystallisation speed can be greatly accelerated  in case of epitaxial growth on suitable substrates \citep{dohnalek2000}. (*fig:crystal_timescale_euclid*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2305.10107"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

209  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
